# FM

In [2]:
%pip install lightfm

  Using cached lightfm-1.17.tar.gz (316 kB)
  ERROR: Command errored out with exit status 1:
   command: /home/roa/universidad/recsys/anime-recommendation/t1recsys/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-8fachine/lightfm/setup.py'"'"'; __file__='"'"'/tmp/pip-install-8fachine/lightfm/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-d17v524_
       cwd: /tmp/pip-install-8fachine/lightfm/
  Complete output (6 lines):
  usage: setup.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
     or: setup.py --help [cmd1 cmd2 ...]
     or: setup.py --help-commands
     or: setup.py cmd --help
  
  error: invalid command 'bdist_wheel'
  ----------------------------------------
  ERROR: Failed building wheel for lightfm
  Running setup.py clean for lightfm
Failed to build lightfm
    Running s

In [2]:
%pip install Cython "numpy<2.0.0" "scipy<=1.13.1" tqdm powerlaw

  Using cached Cython-3.0.12-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
     |████████████████████████████████| 536 kB 22.1 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install recommenders

  Using cached recommenders-1.2.1-py3-none-any.whl (355 kB)
  Using cached retrying-1.3.4-py3-none-any.whl (11 kB)
  Using cached memory_profiler-0.61.0-py3-none-any.whl (31 kB)
     |████████████████████████████████| 1.6 MB 8.6 MB/s eta 0:00:01
  Using cached lightgbm-4.6.0.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Using cached pandera-0.17.2-py3-none-any.whl (203 kB)
  Using cached category_encoders-2.6.4-py2.py3-none-any.whl (82 kB)
  Using cached locust-2.25.0-py3-none-any.whl (1.4 MB)
  Using cached statsmodels-0.14.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.9 MB)
  Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
  Using cached cornac-2.3.0.tar.gz (5.9 MB)
  Using cached transformers-4.46.3-py3-none-any.whl (10.0 MB)
  Using cached numba-0.58.1-cp38-cp38-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.7 MB)
  Using

In [1]:
import os
import sys
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import lightfm
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm import cross_validation
from lightfm.evaluation import precision_at_k as lightfm_prec_at_k
from lightfm.evaluation import recall_at_k as lightfm_recall_at_k

from recommenders.evaluation.python_evaluation import precision_at_k, recall_at_k
from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.models.lightfm.lightfm_utils import (
    track_model_metrics,
    prepare_test_df,
    prepare_all_predictions,
    compare_metric,
    similar_users,
    similar_items,
)
from recommenders.utils.notebook_utils import store_metadata

print("System version: {}".format(sys.version))
print("LightFM version: {}".format(lightfm.__version__))


System version: 3.8.10 (default, Jul 29 2024, 17:02:10) 
[GCC 9.4.0]
LightFM version: 1.17


In [3]:
# Aqui se toma la decision de unir el train.csv con el validation.csv para poder seguir el flujo de
# de trabajo del práctico del curso

# Se instancian los datasets
train_file = pd.read_csv('dataset/train.csv', sep=',', header=0)
test_file = pd.read_csv('dataset/validation.csv', sep=',', header=0)
anime_file = pd.read_csv('dataset/anime.csv', sep=',', header=0)


# Se quitan los registros con rating -1
train_file = train_file[train_file['rating'] != -1]
test_file = test_file[test_file['rating'] != -1]

# Remove the first row from CSV
train_file.to_csv("dataset/train.csv", index=False, header=False)
test_file.to_csv("dataset/validation.csv", index=False, header=False)
anime_file.to_csv("dataset/anime.csv", index=False, header=False)

train_file = pd.read_csv('dataset/train.csv', names = ['userID','itemID','rating'] ,sep=',', header=0)
test_file = pd.read_csv('dataset/validation.csv', names = ['userID','itemID','rating'], sep=',', header=0)

# Se concatenan
data = pd.concat([train_file, test_file], ignore_index=True)

anime_file = pd.read_csv('dataset/anime.csv', names = ['anime_id','name','genre','type','episodes','rating','members'], sep=',', header=0)

data.head()



,userID,itemID,rating
0,31006,8795,7
1,68084,14837,8
2,20881,1536,9
3,25996,1241,8
4,25409,8668,7


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24101 entries, 0 to 24100
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   userID  24101 non-null  int64
 1   itemID  24101 non-null  int64
 2   rating  24101 non-null  int64
dtypes: int64(3)
memory usage: 565.0 KB


In [5]:
DATA_SIZE = '30k'    # Tamaño del dataset (se puede cambiar para recomendar con más datos)

K = 10                          # Tamaño de la recomendación
TEST_PERCENTAGE = 0.25          # Porcentaje usado para set de test
LEARNING_RATE = 0.25
LOSS_FUNCTION = 'bpr'
NO_COMPONENTS = 20              # Factores latentes
NO_EPOCHS = 20                  # Épocas para el entrenamiento
NO_THREADS = 32
ITEM_ALPHA = 1e-6               # Regularizaciones
USER_ALPHA = 1e-6

SEED = 42                       # Semilla para los splits de los datos (se puede eliminar si se desea experimentar)

In [6]:
dataset = Dataset()

dataset.fit(users=data['userID'],
            items=data['itemID'])

# quick check to determine the number of unique users and items in the data
num_users, num_topics = dataset.interactions_shape()
print(f'Num users: {num_users}, num_topics: {num_topics}.')

Num users: 16777, num_topics: 3677.


In [7]:
(interactions, weights) = dataset.build_interactions(data.iloc[:, 0:3].values)

In [8]:
train_interactions, test_interactions = cross_validation.random_train_test_split(
    interactions, test_percentage=TEST_PERCENTAGE,
    random_state=np.random.RandomState(SEED))

In [9]:
print(f"Shape of train interactions: {train_interactions.shape}")
print(f"Shape of test interactions: {test_interactions.shape}")

Shape of train interactions: (16777, 3677)
Shape of test interactions: (16777, 3677)


In [10]:
model1 = LightFM(loss=LOSS_FUNCTION, no_components=NO_COMPONENTS,
                 learning_rate=LEARNING_RATE,
                 random_state=np.random.RandomState(SEED))

In [11]:
model1.fit(interactions=train_interactions,
          epochs=NO_EPOCHS)

In [12]:
uids, iids, interaction_data = cross_validation._shuffle(
    interactions.row, interactions.col, interactions.data,
    random_state=np.random.RandomState(SEED))

cutoff = int((1.0 - TEST_PERCENTAGE) * len(uids))
test_idx = slice(cutoff, None)

In [13]:
uid_map, ufeature_map, iid_map, ifeature_map = dataset.mapping()

In [14]:
with Timer() as test_time:
    test_df = prepare_test_df(test_idx, uids, iids, uid_map, iid_map, weights)
print(f"Took {test_time.interval:.1f} seconds for prepare and predict test data.")
time_reco1 = test_time.interval

Took 1.0 seconds for prepare and predict test data.


In [15]:
test_df.sample(5, random_state=SEED)

,userID,itemID,rating
4366,16497,166,9.0
1129,53290,202,8.0
2602,12431,9741,1.0
1747,53724,2904,10.0
3286,44073,16742,10.0


In [16]:
print(data.columns)


Index(['userID', 'itemID', 'rating'], dtype='object')


In [17]:
data_subset = data.sample(5000)


In [18]:
with Timer() as test_time:
    all_predictions = prepare_all_predictions(data_subset, uid_map, iid_map,
                                              interactions=train_interactions,
                                              model=model1,
                                              num_threads=NO_THREADS)
print(f"Took {test_time.interval:.1f} seconds for prepare and predict all data.")
time_reco2 = test_time.interval

KeyboardInterrupt: 

In [18]:
with Timer() as test_time:
    all_predictions = prepare_all_predictions(data, uid_map, iid_map,
                                              interactions=train_interactions,
                                              model=model1,
                                              num_threads=NO_THREADS)
print(f"Took {test_time.interval:.1f} seconds for prepare and predict all data.")
time_reco2 = test_time.interval

: 